# Item-Based Collaborative Filtering

## Veri seti: https://grouplens.org/datasets/movielens/

### Adım 1: Veri Setinin Hazırlanması
### Adım 2: User Movie Df'inin Oluşturulması
### Adım 3: Item-Based Film Önerilerinin Yapılması
### Adım 4: Çalışma Scriptinin Hazırlanması

In [1]:
######################################
# Adım 1: Veri Setinin Hazırlanması
######################################

In [3]:
import pandas as pd
pd.set_option('display.max_columns', 500)
movie = pd.read_csv('../datasets/movie.csv')
rating = pd.read_csv('../datasets/rating.csv')
df = movie.merge(rating, how="left", on="movieId")
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


In [7]:
df.shape
#Toplam 20000797 tane yorum yapılmış

(20000797, 6)

In [8]:
df["title"].nunique()
#Toplam 27262 tane farklı film var

27262

In [17]:
#1000den az yorum almış filmleri görelim
comment_count=pd.DataFrame(df["title"].value_counts())
comment_count[comment_count["title"]<1000]

,title
One Night at McCool's (2001),999
Rosewood (1997),999
Ted (2012),999
"Bear, The (Ours, L') (1988)",999
Marked for Death (1990),998
...,...
Dr Money and the Boy with No Penis (2004),1
Nantucket Film Festival's Comedy Roundtable (2012),1
Fiend with the Electronic Brain (Blood of Ghastly Horror) (1972),1
Divine Horsemen: The Living Gods of Haiti (1985),1


In [6]:
######################################
# Adım 2: User Movie Df'inin Oluşturulması
######################################

In [22]:
#1000den az yorum almış filmler topluluk tarafından çok izlenmediği yada yorum yapılmaya gerek duyulmadığı için kaldıralım
rare_movies=comment_count[comment_count["title"]<1000].index
common_movies=df[~df["title"].isin(rare_movies)]
common_movies["title"].nunique()
#27262 filmden geriye 3159 film kalmış

3159

In [31]:
df["userId"].value_counts().mean()
#Kullanıcılar ortalama 128 yorum yapmış aykırı değerler de olduğunu düşünerek
#60'ın altında yorum yapan kullanıcıları kaldıralım

128.2809600485223

In [34]:
user_count=pd.DataFrame(df["userId"].value_counts())
rare_user=user_count[user_count["userId"]<60].index
common_movies_common_users=common_movies[~common_movies["userId"].isin(rare_user)]

In [37]:
common_movies_common_users["userId"].value_counts()

8405.0      2966
118205.0    2808
74142.0     2580
34576.0     2479
125794.0    2460
            ... 
86704.0       60
35953.0       60
70406.0       60
103170.0      60
55024.0       60
Name: userId, Length: 72854, dtype: int64

In [38]:
#Satırlarda kullanacılar sütunlarda ise filmler olacak şekilde yeni bir dataframe oluşturalım
user_movie_df = common_movies_common_users.pivot_table(index=["userId"], columns=["title"], values="rating")
user_movie_df

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),16 Blocks (2006),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),2046 (2004),21 (2008),21 Grams (2003),21 Jump Street (2012),24 Hour Party People (2002),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),3 Ninjas (1992),3-Iron (Bin-jip) (2004),30 Days of Night (2007),300 (2007),3000 Miles to Graceland (2001),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",42 Up (1998),48 Hrs. (1982),50 First Dates (2004),50/50 (2011),54 (1998),"6th Day, The (2000)","7th Voyage of Sinbad, The (1958)",8 1/2 (8½) (1963),8 Heads in a Duffel Bag (1997),8 Mile (2002),8MM (1999),9 (2009),9 1/2 Weeks (Nine 1/2 Weeks) (1986),"A-Team, The (2010)",A.I. Artificial Intelligence (2001),AVP: Alien vs. Predator (2004),AVPR: Aliens vs. Predator - Requiem (2007),About Last Night... (1986),About Schmidt (2002),About a Boy (2002),"Absent-Minded Professor, The (1961)",Absolute Power (1997),"Abyss, The (1989)",Accepted (2006),"Accidental Tourist, The (1988)","Accused, The (1988)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Across the Universe (2007),Adaptation (2002),Addams Family Values (1993),"Addams Family, The (1991)",Addicted to Love (1997),"Adjustment Bureau, The (2011)",Adventureland (2009),Adventures in Babysitting (1987),"Adventures of Baron Munchausen, The (1988)","Adventures of Buckaroo Banzai Across the 8th Dimension, The (1984)","Adventures of Milo and Otis, The (Koneko monogatari) (1986)","Adventures of Pinocchio, The (1996)","Adventures of Priscilla, Queen of the Desert, The (1994)","Adventures of Robin Hood, The (1938)","Adventures of Rocky and Bullwinkle, The (2000)","Adventures of Tintin, The (2011)",Aeon Flux (2005),"Affair to Remember, An (1957)",Affliction (1997),"African Queen, The (1951)",After Hours (1985),"Age of Innocence, The (1993)",Agnes of God (1985),"Aguirre: The Wrath of God (Aguirre, der Zorn Gottes) (1972)",Air America (1990),Air Bud (1997),Air Force One (1997),Airheads (1994),Airplane II: The Sequel (1982),Airplane! (1980),Airport (1970),Akira (1988),Aladdin (1992),Aladdin and the King of Thieves (1996),Alexander (2004),Alfie (2004),Ali (2001),Alice in Wonderland (1951),Alice in Wonderland (2010),Alien (1979),Alien Nation (1988),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Alive (1993),All About Eve (1950),All About My Mother (Todo sobre mi madre) (1999),All Dogs Go to Heaven (1989),All Dogs Go to Heaven 2 (1996),All Quiet on the Western Front (1930),All That Jazz (1979),All the President's Men (1976),All the Pretty Horses (2000),Almost Famous (2000),Along Came Polly (2004),Along Came a Spider (2001),"Alphaville (Alphaville, une étrange aventure de Lemmy Caution) (1965)",Altered States (1980),Always (1989),Amadeus (1984),Amateur (1994),"Amazing Spider-Man, The (2012)","Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",America's Sweethearts (2001),American Beauty (1999),American Gangster (2007),American Gigolo (1980),American Graffiti (1973),American History X (1998),American Hustle (2013),American Movie (1999),American Pie (1999),American Pie 2 (2001),"American President, The (1995)",American Psycho (2000),American Splendor (2003),"American Tail, An (1986)","American Tail: Fievel Goes West, An (1991)",Amer

In [39]:
######################################
# Adım 3: Item-Based Film Önerilerinin Yapılması
######################################

In [41]:
#Rastgele bir film seçip diğer filmler ile olan korelasyonuna bakalım
#Korelasyon ne kadar yüksek ise filmler o kadar benzerdir
movie_name = "Matrix, The (1999)"
movie_points = user_movie_df[movie_name]
user_movie_df.corrwith(movie_points).sort_values(ascending=False).head(10)
#Görüldüğü gibi tahminler gayet başarılı

title
Matrix, The (1999)                                           1.000000
Matrix Reloaded, The (2003)                                  0.512076
Matrix Revolutions, The (2003)                               0.443333
Animatrix, The (2003)                                        0.362384
Edge of Tomorrow (2014)                                      0.341580
Blade (1998)                                                 0.332560
Terminator 2: Judgment Day (1991)                            0.332034
Minority Report (2002)                                       0.328452
Mission: Impossible (1996)                                   0.321441
Lord of the Rings: The Fellowship of the Ring, The (2001)    0.316540
dtype: float64

In [58]:
#Başka bir film gözlemleyelim
movie_name = 'Batman (1989)'
movie_points = user_movie_df[movie_name]
user_movie_df.corrwith(movie_points).sort_values(ascending=False).head(10)
#Diğer batman filmlerinin üstte gelmesi tahminlerin başarısını gösteriyor çünkü film isimleriyle hiç ilgilenmedik

title
Batman (1989)                                 1.000000
Batman Returns (1992)                         0.524515
Batman: Mask of the Phantasm (1993)           0.424684
Ghostbusters (a.k.a. Ghost Busters) (1984)    0.386644
Superman (1978)                               0.368373
Beetlejuice (1988)                            0.362843
Who Framed Roger Rabbit? (1988)               0.352950
Indiana Jones and the Last Crusade (1989)     0.348846
Dick Tracy (1990)                             0.342189
Dragon: The Bruce Lee Story (1993)            0.330676
dtype: float64

In [61]:
#Girilen anahtar kelimeyi barındıran film isimlerini getiren bir fonksiyon yazalım
def check_film(keyword, user_movie_df):
    return [col for col in user_movie_df.columns if keyword in col]
movie_name=check_film("Titanic",user_movie_df)
movie_name

['Titanic (1953)', 'Titanic (1997)']

In [62]:
movie_points = user_movie_df[movie_name[0]]
print(user_movie_df.corrwith(movie_points).sort_values(ascending=False).head(5))
movie_points = user_movie_df[movie_name[1]]
print("****************************************************************")
print(user_movie_df.corrwith(movie_points).sort_values(ascending=False).head(5))

title
Titanic (1953)                      1.000000
Public Enemies (2009)               0.749944
Oblivion (2013)                     0.715474
Titanic (1997)                      0.676414
Jimmy Neutron: Boy Genius (2001)    0.668053
dtype: float64
****************************************************************
title
Titanic (1997)           1.000000
Titanic (1953)           0.676414
Ghost (1990)             0.462839
Pearl Harbor (2001)      0.446844
Bodyguard, The (1992)    0.432937
dtype: float64


In [63]:
######################################
# Adım 4: Çalışma Scriptinin Hazırlanması
######################################

In [64]:
def create_user_movie_df():
    import pandas as pd
    movie = pd.read_csv('../datasets/movie.csv')
    rating = pd.read_csv('../datasets/rating.csv')
    df = movie.merge(rating, how="left", on="movieId")
    comment_counts = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 1000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df

def item_based_recommender(movie_name, user_movie_df):
    movie_name = user_movie_df[movie_name]
    return user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

In [ ]:
#Random bir film seçip bununla alakalı tavsiyelere bakalım
user_movie_df=create_user_movie_df()
movie_name = pd.Series(user_movie_df.columns).sample(1).values[0]
item_based_recommender(movie_name, user_movie_df)